In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# ===== ENV =====
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

llm = ChatGroq(model="llama-3.3-70b-versatile")

result = llm.invoke("What is 2 + 2?")
result


e:\anaconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='2 + 2 = 4.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 43, 'total_tokens': 52, 'completion_time': 0.012050508, 'completion_tokens_details': None, 'prompt_time': 0.001169999, 'prompt_tokens_details': None, 'queue_time': 0.058848731, 'total_time': 0.013220507}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019be3b6-c2ce-74e2-92a8-9f51ed02e434-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 43, 'output_tokens': 9, 'total_tokens': 52})

In [2]:
from typing import Annotated,List
import operator
from typing_extensions import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage,SystemMessage
from typing_extensions import TypedDict

In [3]:
class Section(BaseModel):
  name:str= Field(description="Name for this section report")
  description:str= Field(description="Brief Overview of the main topics and concepts of the section")

class Sections(BaseModel):
  sections:List[Section] = Field(
    description= "Sections of the report"
  )
planer = llm.with_structured_output(Sections)


In [6]:
class State(TypedDict):
  topic:str
  completed_sections :Annotated[list, operator.add]
  final_report :str

class WorkeState(TypedDict):
  section: Section
  completed_sections :Annotated[list, operator.add]



In [ ]:
def orchestrator(state:State):
  """Orchestrator function to manage the workflow of report generation."""

  report_sections = planer.invoke(
    [
      SystemMessage(content="You are an expert report planer."),
      HumanMessage(content=f"Create a detailed report plan with sections on the topic: {state['topic']}")
    ]
  )
  print("Report Sections Planed:", report_sections)
  return {"sections": report_sections.sections}